# T1055.002 - Portable Executable Injection
Adversaries may inject portable executables (PE) into processes in order to evade process-based defenses as well as possibly elevate privileges. PE injection is a method of executing arbitrary code in the address space of a separate live process. 

PE injection is commonly performed by copying code (perhaps without a file on disk) into the virtual address space of the target process before invoking it via a new thread. The write can be performed with native Windows API calls such as <code>VirtualAllocEx</code> and <code>WriteProcessMemory</code>, then invoked with <code>CreateRemoteThread</code> or additional code (ex: shellcode). The displacement of the injected code does introduce the additional requirement for functionality to remap memory references. (Citation: Elastic Process Injection July 2017) 

Running code in the context of another process may allow access to the process's memory, system/network resources, and possibly elevated privileges. Execution via PE injection may also evade detection from security products since the execution is masked under a legitimate process. 

## Atomic Tests

### Atomic Test #1 - Portable Executable Injection
This test injects a portable executable into a remote Notepad process memory using Portable Executable Injection and base-address relocation techniques. When successful, a message box will appear with the title "Warning" and the content "Atomic Red Team" after a few seconds.
**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Portable Executable to inject must exist at specified location (#{exe_binary})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1055.002\bin\RedInjection.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1055.002\bin\RedInjection.exe") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1055.002/bin/RedInjection.exe" -OutFile "PathToAtomicsFolder\T1055.002\bin\RedInjection.exe"

```

In [ ]:
Invoke-AtomicTest T1055.002 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Start-Process "PathToAtomicsFolder\T1055.002\bin\RedInjection.exe"
Start-Sleep -Seconds 7
Get-Process -Name Notepad -ErrorAction SilentlyContinue | Stop-Process -Force```

In [ ]:
Invoke-AtomicTest T1055.002 -TestNumbers 1

#### Cleanup: 
```powershell
Get-Process -Name Notepad -ErrorAction SilentlyContinue | Stop-Process -Force```

In [ ]:
Invoke-AtomicTest T1055.002 -TestNumbers 1 -Cleanup

## Detection
Monitoring Windows API calls indicative of the various types of code injection may generate a significant amount of data and may not be directly useful for defense unless collected under specific circumstances for known bad sequences of calls, since benign use of API functions may be common and difficult to distinguish from malicious behavior. Windows API calls such as <code>CreateRemoteThread</code> and those that can be used to modify memory within another process, such as <code>VirtualAllocEx</code>/<code>WriteProcessMemory</code>, may be used for this technique.(Citation: Elastic Process Injection July 2017)

Analyze process behavior to determine if a process is performing actions it usually does not, such as opening network connections, reading files, or other suspicious actions that could relate to post-compromise behavior. 